In [3]:
# import the libraries

import numpy as np
import pandas as pd
import platform

root_path = "C:\\Users\\mrbal\\Documents\\NLP\\RL\\basic_reinforcement_learning"
info_path = ""
sep = '/'
if platform.system() == "Windows":
    info_path = root_path + "\\NLP_datasets\\RT_Polarity\\data_info_bert_windows.json"
    sep = '\\'
    print("Running on windows...")
else:
    root_path = "/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning"
    info_path = root_path + "/NLP_datasets/RT_Polarity/data_info_bert_macos.json"

import sys
sys.path.append(root_path)
from RL_for_NLP.text_environments import TextEnvClfWithBertTokens, TextEnvClf, TextEnvClfForBertModels
from RL_for_NLP.text_reward_functions import calculate_stats_from_cm

from RL_for_NLP.text_data_pools import PartialReadingDataPoolWithTokens, PartialReadingDataPoolWithBertTokens
import NLP_utils.preprocessing as nlp_processing
import reinforce_algorithm_utils as rl_monte_carlo
import policy_networks as pn

import torch
from torch.optim import Adam
import mlflow
from RL_for_NLP.text_reward_functions import calculate_stats_from_cm


from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3 import DQN

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import json
from collections import Counter
from tqdm import tqdm

In [4]:
with open(info_path, "r") as f:
    data_info = json.load(f)


data_train = nlp_processing.openDfFromPickle(data_info["path"] + sep + "rt-polarity-train-bert.pkl")
print(data_train.shape)
data_train.head()

(8528, 7)


,Unnamed: 0,label,review,label_str,review_bert_input_ids,review_bert_token_type_ids,review_bert_attention_mask
0,5829,1,does what fine documentary does best it extend...,good,"[101, 1674, 1184, 2503, 4148, 1674, 1436, 1122...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,3209,0,it drowns in sap,bad,"[101, 1122, 22592, 1116, 1107, 21718, 1643, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,4323,0,the scriptwriters are no less menace to societ...,bad,"[101, 1103, 5444, 13814, 1116, 1132, 1185, 175...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,281,0,there an excellent minute film here unfortunat...,bad,"[101, 1175, 1126, 6548, 2517, 1273, 1303, 1667...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
4,100,0,narratively trouble every day is plodding mess,bad,"[101, 8195, 1193, 3819, 1451, 1285, 1110, 185,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."


In [5]:
# declare some hyperparameters
WINDOW_SIZE = 16
MAX_STEPS = int(1e+5)
TRAIN_STEPS = int(1e+3)
VOCAB_SIZE = data_info["vocab_size"]
REWARD_FN = "score"
print(f"Vocab size: {VOCAB_SIZE}")

Vocab size: 28996


In [6]:
data_test = nlp_processing.openDfFromPickle(data_info["path"] + sep + "rt-polarity-test-bert.pkl")
data_val = nlp_processing.openDfFromPickle(data_info["path"] + sep + "rt-polarity-val-bert.pkl")

train_pool = PartialReadingDataPoolWithBertTokens(data_train, "review", "label", WINDOW_SIZE, mask=False)
test_pool = PartialReadingDataPoolWithBertTokens(data_test, "review", "label", WINDOW_SIZE, mask=False)
val_pool = PartialReadingDataPoolWithBertTokens(data_val, "review", "label", WINDOW_SIZE, mask=False)

Maximum sentence length in pool: 50


Padding the data and populating samples...: 100%|██████████| 8528/8528 [00:00<00:00, 26297.10it/s]


Maximum sentence length in pool: 50


Padding the data and populating samples...: 100%|██████████| 1067/1067 [00:00<00:00, 30675.16it/s]


Maximum sentence length in pool: 50


Padding the data and populating samples...: 100%|██████████| 1066/1066 [00:00<00:00, 26552.53it/s]


In [10]:
train_env = TextEnvClfWithBertTokens(train_pool, VOCAB_SIZE, MAX_STEPS, "score", False)
val_env = TextEnvClfWithBertTokens(val_pool, VOCAB_SIZE, 2000, "score", False)
test_env = TextEnvClfWithBertTokens(test_pool, VOCAB_SIZE, 2000, "score", False)
print("All environments are created.")

---- Inside constructor ---------
State shape/type:  (16,) <class 'numpy.ndarray'>
------------- End Constructor-------------
---- Inside constructor ---------
State shape/type:  (16,) <class 'numpy.ndarray'>
------------- End Constructor-------------
---- Inside constructor ---------
State shape/type:  (16,) <class 'numpy.ndarray'>
------------- End Constructor-------------
All environments are created.


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}.")

Using device: cpu.


In [9]:
def eval_model(model, env, total_timesteps=10000):
    done = False
    obs = env.reset()
    total_reward = 0.0
    actions = []
    seen_samples = 0
    for _ in tqdm(range(total_timesteps)):
        action, _states = model.predict(obs)
        action = action.item()
        obs, rewards, done, info = env.step(action)
        action = env.action_space.ix_to_action(action)
        if action in env.pool.possible_actions:
            seen_samples += 1
        if done:
            obs = env.reset()
        actions.append(action)
        total_reward += rewards
    print("---------------------------------------------");
    print(f"Total Steps and seen samples: {len(actions), seen_samples}")
    print(f"Total reward: {total_reward}")
    print(f"Stats:  {calculate_stats_from_cm(env.confusion_matrix)}")
    acts = list(Counter(actions).keys())
    freqs = list(Counter(actions).values())
    total = len(actions)
    print(f"Action stats --  {[{acts[ii]: freqs[ii]/total} for ii in range(len(acts))]}")
    print("---------------------------------------------")

In [11]:
policy_kwargs = dict(
    features_extractor_class=pn.RNNExtractor,
    features_extractor_kwargs=dict(vocab_size = VOCAB_SIZE, embed_dim = 5,
                 rnn_type = "gru", rnn_hidden_size = 2, rnn_hidden_out = 2, rnn_bidirectional = True,
                 features_dim = 10, units = 10),
)


model = DQN("MlpPolicy", train_env, policy_kwargs=policy_kwargs, verbose=1, batch_size=64, gamma=1.)
for i in range(int(15)):
    model.learn(total_timesteps=int(1e+4)*15, reset_num_timesteps=True, progress_bar=True)
    print("======= On train env: ===============")
    eval_model(model, train_env)
    print("======= On val env: ===============")
    eval_model(model, val_env)

Output()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


======= On train env: ===============


100%|██████████| 10000/10000 [00:12<00:00, 799.76it/s]


---------------------------------------------
Total Steps and seen samples: (10000, 9797)
Total reward: 9427.780000000035
Stats:  {'accuracy': 0.8643023395660449, 'precision': 0.8147768538162334, 'recall': 0.8595807517723904, 'f1': 0.8318673587498024}
Action stats --  [{'bad': 0.2505}, {'good': 0.7292}, {'<next>': 0.0203}]
---------------------------------------------
======= On val env: ===============


...

---------------------------------------------
Total Steps and seen samples: (10000, 9834)
Total reward: 9495.91000000003
Stats:  {'accuracy': 0.8686360557157585, 'precision': 0.8199084868959574, 'recall': 0.8674882291452275, 'f1': 0.8377985734682387}
Action stats --  [{'bad': 0.2548}, {'good': 0.7286}, {'<next>': 0.0166}]
---------------------------------------------
======= On val env: ===============


100%|██████████| 10000/10000 [00:13<00:00, 738.64it/s]

Output()

---------------------------------------------
Total Steps and seen samples: (10000, 9817)
Total reward: -6342.919999999974
Stats:  {'accuracy': 0.2666951358386205, 'precision': 0.5410722457690277, 'recall': 0.5053613428259242, 'f1': 0.22758900172688548}
Action stats --  [{'good': 0.9563}, {'bad': 0.0254}, {'<next>': 0.0183}]
---------------------------------------------


KeyboardInterrupt: 

**Reinforce Algorithm**

In [8]:
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import reinforce_algorithm_utils as rl_monte_carlo
import gym
import torch
from torch.optim import Adam

import imageio

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}.")

Using device: cpu.


In [10]:
print(train_env.current_state_input_id.shape)
s_size = train_env.current_state_input_id.shape[0]
a_size = len(train_env.action_space)
print(s_size, a_size)

(8,)
8 4


In [11]:
hyperparameters = {
    "h_sizes": [64, 32],
    "n_training_episodes": 100000,
    "n_evaluation_episodes": 10,
    "max_t": 25,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": None,
    "state_space": s_size,
    "action_space": a_size,
}

In [12]:
# Create policy and place it to the device
import policy_networks as pn
"""policy = pn.Transformer_Baseline_Policy(data_info["vocab_size"], hyperparameters["state_space"], hyperparameters["action_space"],
                                        num_heads=1, num_layers=1)"""
policy = pn.RNN_Baseline_Policy(data_info["vocab_size"], hyperparameters["state_space"], hyperparameters["action_space"])
optimizer = Adam(policy.parameters(), lr=hyperparameters["lr"])

In [51]:
params = dict(policy.named_modules())
print(params)


{'': RNN_Baseline_Policy(
  (embed_enc): Embedding(28996, 5, max_norm=True)
  (rnn): GRU(5, 2, num_layers=2, batch_first=True, bidirectional=True)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=32, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=4, bias=True)
), 'embed_enc': Embedding(28996, 5, max_norm=True), 'rnn': GRU(5, 2, num_layers=2, batch_first=True, bidirectional=True), 'flat': Flatten(start_dim=1, end_dim=-1), 'fc1': Linear(in_features=32, out_features=50, bias=True), 'fc2': Linear(in_features=50, out_features=50, bias=True), 'fc3': Linear(in_features=50, out_features=4, bias=True)}


In [55]:
import mlflow
import json

params_s = str(params)
mlflow.log_dict(params_s, "params.json")

In [ ]:
scores = rl_monte_carlo.reinforce_algorithm(train_env, policy,
                   optimizer,
                   hyperparameters["n_training_episodes"], 
                   hyperparameters["max_t"],
                   hyperparameters["gamma"])

In [18]:
import torch
import torch.nn as nn

x = torch.Tensor([[1, 3, 5, 7, 9], [2, 4, 6, 8, 10]]).int()
emb = nn.Embedding(20, 5)
lstm = nn.LSTM(5, 11, 7, bidirectional=True, batch_first=True)
print(x.shape)
x = emb(x)
print(x)
print(x.shape)
x, (h, _) = lstm(x)
print(x)
print(x.shape, h.shape)

torch.Size([2, 5])
tensor([[[-0.4974, -0.1800,  0.1256, -0.0997, -2.6214],
         [-1.6004,  0.7747,  0.3733,  1.3647,  0.0687],
         [-0.7068,  0.6754,  0.6912,  1.7885, -0.4849],
         [-1.1904, -0.5578,  1.5177, -0.2507, -0.8633],
         [ 1.2573,  0.4593,  1.2733,  0.4671,  0.1200]],

        [[-1.1528,  2.0933, -0.6916, -0.2654,  1.5681],
         [-1.5847, -0.2593,  0.0507,  0.8566, -0.7230],
         [-0.7984, -0.9824, -0.1521,  0.4700, -1.4515],
         [ 1.7227,  0.5693,  0.6390, -0.5363, -0.9067],
         [ 1.0751, -0.5434,  0.9034, -0.1167,  1.8581]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 5, 5])
tensor([[[-1.2247e-01, -8.0512e-02, -3.4734e-02, -2.2318e-03,  7.6149e-04,
           1.1641e-01, -1.8003e-02,  5.5952e-02,  1.7178e-02, -4.6703e-02,
           2.3925e-03,  1.3036e-01,  1.6704e-01,  1.1473e-01, -7.4383e-02,
           5.7412e-02,  8.1825e-02, -4.2972e-02,  9.8029e-03, -1.9270e-01,
           5.2298e-02,  1.0932e-01],
         [-1.6308e-01

In [ ]:
-1e-1*(np.log2(500000))

-1.8931568569324175

In [ ]:
rl_monte_carlo.evaluate_agent(train_env, 10, 100, policy)

In [ ]:
rl_monte_carlo.evaluate_agent(val_env, 10, 100, policy)

In [ ]:
rl_monte_carlo.evaluate_on_clf(val_env, policy, pos_label="good")

In [ ]:
def eval_model(model, env):
    done = False
    obs = env.reset()
    total_reward = 0.0
    actions = []
    while not done:
        action, _states = model.predict(obs)
        action = action.item()
        obs, rewards, done, info = env.step(action)
        actions.append(env.action_space.ix_to_action(action))
        total_reward += rewards
    print("---------------------------------------------")
    print(f"Predicted Label {actions}")
    print(f"Oracle Label: {env.current_label}")
    print(f"Total Reward: {total_reward}")
    print("---------------------------------------------")


In [ ]:
eval_model(policy, val_env)

In [ ]:
import mlflow

In [ ]:

mlflow.set_tracking_uri("file:/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning/agent_models/rt-polarity")
mlflow.set_experiment("rnn")
mlflow.start_run()
mlflow.log_param("n_episodes", 500)
mlflow.pytorch.log_model(policy, "rnn-default")
mlflow.end_run()

In [ ]:
mlflow.end_run()

In [ ]:
mlflow.set_tracking_uri("file:/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning/agent_models/rt-polarity")
uri = "runs:/9f09e3fa6df6427ba2889e89ac787bcc/rnn-default"

model = mlflow.pytorch.load_model(uri)

In [ ]:
rl_monte_carlo.evaluate_agent(env, 10, 100, model)

In [ ]:
rl_monte_carlo.evaluate_on_clf(env, model)

In [ ]:
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3 import DQN

model = DQN(policy=MlpPolicy, env=train_env, learning_rate=0.001, batch_size=3)

for i in range(int(5)):
    model.learn(total_timesteps=int(1e+3), reset_num_timesteps=False)
    eval_model(model, val_env)

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Get the state space and action space
s_size = train_env.observation_space.shape[0]
a_size = len(train_env.action_space.actions)
print(s_size, a_size)

In [ ]:
model = A2C(policy = "MlpPolicy",
            env = train_env,
            gae_lambda = 0.9,
            gamma = 0.99,
            learning_rate = 0.00096,
            max_grad_norm = 0.5,
            n_steps = 8,
            vf_coef = 0.4,
            ent_coef = 0.0,
            tensorboard_log = "./tensorboard",
            policy_kwargs=dict(
            log_std_init=-2, ortho_init=False),
            normalize_advantage=False,
            use_rms_prop= True,
            verbose=1)

In [ ]:
for i in range(3):
    model.learn(500, log_interval=250)
    print(evaluate_policy(model, val_env, 3))

In [ ]:
evaluate_policy(model, val_env, n_eval_episodes=1)

In [ ]:
rl_monte_carlo.evaluate_on_clf(val_env, model)